Based on the study from [here](http://large.stanford.edu/courses/2017/ph240/perry2/docs/grogg-13apr05.pdf), we see that the weight of one wind turbine blade (tons) $\approx \frac{1}{6} \times $ rotor weight (tons).

So, if we have rotor weight, we can make a precise estimation of the blade weight. Unfortunately our New York State Data, doesn't have rotor weight. Thus, we need to create a predictive model for it.

We can use The Wind Power dataset linked [here](https://www.thewindpower.net/store_manufacturer_turbine_en.php?id_type=4). (Reached out to the founder, and he shared some sample data for our project 🐐)

# Imports

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

# sklearn model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# sklearn pipeline  
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline

# sklearn preprocessing
from sklearn.preprocessing import OneHotEncoder

# Models
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor

# Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error

# Save Models
import joblib

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Load Development Data

In [ ]:
df = pd.read_csv('../../data/raw/the_wind_power_sample/turbine_data.csv',
                 header=[0,1],
                 na_values = '#ND')

## Find Relevant Columns

In [ ]:
df.columns

MultiIndex([(                'ID',            '#ND = no data'),
            (              'Name',       'Unnamed: 1_level_1'),
            (      'Manufucturer',       'Unnamed: 2_level_1'),
            (          'Offshore',       'Unnamed: 3_level_1'),
            (       'Rated power',                       'kW'),
            (    'Rotor diameter',                        'm'),
            (        'Swept area',                       'm2'),
            (     'Specific area',                    'm2/kW'),
            (  'Number of blades',       'Unnamed: 8_level_1'),
            ('Minimum hub height',                        'm'),
            ('Maximum hub height',                        'm'),
            (    'Nacelle weight',                     'Tons'),
            (      'Tower weight',                     'Tons'),
            (      'Rotor weight',                     'Tons'),
            (      'Total weight',                     'Tons'),
            (   'Available since', 'Form

We can only use columns that appear in the USGS data as predictors. Thus, we can use the following columns:


The Wind Power Data | USGS Turbine Data
--- | ---
Manufucturer | t_manu
Rated power (kW) | t_cap (kW)
Minimum hub height (meters) ; Maximum hub height (meters) | t_hh (meters)
Rotor diameter (meters) | t_rd (meters)
Swept area (meters$^2$) | t_rsa (meters$^2$)
Minimum hub height (meters) ; Maximum hub height (meters) ; Rotor diameter (meters) | t_ttlh (meters)
Rotor weight (Tons) | **To be predicted**

In [ ]:
relevant_columns = ['Manufucturer','Rated power', 'Minimum hub height', 'Maximum hub height',
                    'Rotor diameter', 'Swept area', 'Rotor weight']

df = pd.read_csv('../../data/raw/the_wind_power_sample/turbine_data.csv', 
                 na_values = '#ND',usecols=relevant_columns,skiprows=[1])
df.head(3)

,Manufucturer,Rated power,Rotor diameter,Swept area,Minimum hub height,Maximum hub height,Rotor weight
0,Acciona,1500,70.0,3848.5,60.0,80.0,15
1,Acciona,1500,77.0,4656.6,60.0,80.0,15
2,Acciona,1500,82.0,5281.0,80.0,80.0,15


## Deal with Missing Values

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Manufucturer        467 non-null    object 
 1   Rated power         467 non-null    int64  
 2   Rotor diameter      467 non-null    float64
 3   Swept area          467 non-null    float64
 4   Minimum hub height  448 non-null    float64
 5   Maximum hub height  448 non-null    float64
 6   Rotor weight        467 non-null    int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 25.7+ KB


Let's drop the rows with NA values. It looks like we would still have 448 rows after dropping; this is not significantly less than the 467 we have currently.

In [ ]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 448 entries, 0 to 466
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Manufucturer        448 non-null    object 
 1   Rated power         448 non-null    int64  
 2   Rotor diameter      448 non-null    float64
 3   Swept area          448 non-null    float64
 4   Minimum hub height  448 non-null    float64
 5   Maximum hub height  448 non-null    float64
 6   Rotor weight        448 non-null    int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 28.0+ KB


## Is it Worth Keeping the Manufacturer Column?

If the USGS data and The Wind Power Data consist of (mostly) the same manufacturers, it is worth including.


In [ ]:
useful_columns = ['case_id','t_state','t_county','t_fips','p_name','p_year',
                  'p_tnum','p_cap','t_manu','t_model','t_cap','t_hh','t_rd',
                  't_rsa','t_ttlh','xlong','ylat']

USGS_data = pd.read_csv('../../data/raw/usgs/usgs_data.csv',
                 engine='python',encoding='latin1',usecols=useful_columns)
USGS_data = USGS_data[USGS_data.t_state=='NY']

USGS data has 14 unique manufacturers. The Wind Power data has 110. Here are the manufacturers that seem to be missing from The Wind Power dataset:

In [ ]:
USGS_manufacturers = list(USGS_data.t_manu.unique())
USGS_manu_list = [x.lower() if type(x)==str else x for x in USGS_manufacturers]

TWP_manufacturers = list(df.Manufucturer.unique())
TWP_manu_list = [x.lower() for x in TWP_manufacturers]

for manu in USGS_manu_list:
    if manu not in TWP_manu_list:
        print(f"{manu} is in the USGS data, but not in The Wind Power datset")

siemens gamesa renewable energy is in the USGS data, but not in The Wind Power datset
ge wind is in the USGS data, but not in The Wind Power datset
clipper is in the USGS data, but not in The Wind Power datset
northern power systems is in the USGS data, but not in The Wind Power datset
fuhrlander is in the USGS data, but not in The Wind Power datset
hyundai is in the USGS data, but not in The Wind Power datset
nan is in the USGS data, but not in The Wind Power datset


Clipper, Nothern Power Systems, and Hyundai are legitimately missing from The Wind Power dataset. There is enough overlap between USGS data and The Wind Power data, that it is worth keeping the Manufacturer.

We will do some pre-processing so the remainder of the classes align. Most importantly, we will replace many of the categories with 'Other' to reduce the cardinality of the variable.

In [ ]:
manufacturer_processed = []
for i in list(df.Manufucturer):
    j = i.lower()
    if ('siemens' in j) or ('gamesa' in j):
        manufacturer_processed.append('siemens gamesa renewable energy')
    elif j == 'ge energy':
        manufacturer_processed.append('ge wind')
    elif j=='fuhrländer':
        manufacturer_processed.append('fuhrlander')
    elif j in USGS_manu_list:
        manufacturer_processed.append(j)
    else:
        manufacturer_processed.append('other')

In [ ]:
df.insert(1, 'manufacturer_processed', manufacturer_processed)

The cardinality of our new manufacturer column is 10. We will be able to use One-Hot encoding if needed.

In [ ]:
df.manufacturer_processed.nunique()

10

In [ ]:
processed_manu = list(df.manufacturer_processed.unique())
processed_manu.sort()
processed_manu

['fuhrlander',
 'ge wind',
 'goldwind',
 'nordex',
 'other',
 'repower',
 'siemens gamesa renewable energy',
 'vensys',
 'vergnet',
 'vestas']

## Derive hub_height

In [ ]:
df['hub_height'] = (df['Minimum hub height'] + df['Maximum hub height'])/2

In [ ]:
df.head()

,Manufucturer,manufacturer_processed,Rated power,Rotor diameter,Swept area,Minimum hub height,Maximum hub height,Rotor weight,hub_height
0,Acciona,other,1500,70.0,3848.5,60.0,80.0,15,70.00
1,Acciona,other,1500,77.0,4656.6,60.0,80.0,15,70.00
2,Acciona,other,1500,82.0,5281.0,80.0,80.0,15,80.00
3,Acciona,other,3000,100.0,7854.0,100.0,120.0,66,110.00
4,Acciona,other,3000,109.0,9331.3,95.5,120.0,66,107.75


## Rename columns

In [ ]:
df.columns = ['manufacturer','manufacturer_processed','rated_power','rotor_diameter',
              'swept_area','min_hub_height','max_hub_height','rotor_weight','hub_height']

In [ ]:
df.head()

,manufacturer,manufacturer_processed,rated_power,rotor_diameter,swept_area,min_hub_height,max_hub_height,rotor_weight,hub_height
0,Acciona,other,1500,70.0,3848.5,60.0,80.0,15,70.00
1,Acciona,other,1500,77.0,4656.6,60.0,80.0,15,70.00
2,Acciona,other,1500,82.0,5281.0,80.0,80.0,15,80.00
3,Acciona,other,3000,100.0,7854.0,100.0,120.0,66,110.00
4,Acciona,other,3000,109.0,9331.3,95.5,120.0,66,107.75


# Modeling

## Random Forest

We start with a random forest for two main reasons. First, it is easy to interpret feature importance after building a Random Forest. Second, it is a powerful model, so it may achieve a great baseline.

In [ ]:
df.head()

,manufacturer,manufacturer_processed,rated_power,rotor_diameter,swept_area,min_hub_height,max_hub_height,rotor_weight,hub_height
0,Acciona,other,1500,70.0,3848.5,60.0,80.0,15,70.00
1,Acciona,other,1500,77.0,4656.6,60.0,80.0,15,70.00
2,Acciona,other,1500,82.0,5281.0,80.0,80.0,15,80.00
3,Acciona,other,3000,100.0,7854.0,100.0,120.0,66,110.00
4,Acciona,other,3000,109.0,9331.3,95.5,120.0,66,107.75


Split dataset into predictors and output.

In [ ]:
X = np.array(df[['manufacturer_processed','rated_power','rotor_diameter','swept_area','hub_height']])
y = np.array(df['rotor_weight'])

numeric_features = [1,2,3,4]
categorical_features = [0]

In [ ]:
categorical_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers = [
        ("cat",categorical_transformer,categorical_features),
        ("num","passthrough",numeric_features)
    ]
)

pipe = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',RandomForestClassifier(random_state=10282022, n_jobs = -1))
])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=10282022)

In [ ]:
pipe.fit(X_train, y_train)
y_pred_train = pipe.predict(X_train)
y_pred_test = pipe.predict(X_test)


print(f"Without hyperparameter tuning...")

results = pd.DataFrame({'Metric (tons)':['mean_absolute_error', 'mean_squared_error', 'median_absolute_error'],
              'Train':[mean_absolute_error(y_train,y_pred_train),
                       mean_squared_error(y_train,y_pred_train),
                       median_absolute_error(y_train,y_pred_train)],
              'Test':[mean_absolute_error(y_test,y_pred_test),
                       mean_squared_error(y_test,y_pred_test),
                       median_absolute_error(y_test,y_pred_test)]})
display(results)

Without hyperparameter tuning...


,Metric (tons),Train,Test
0,mean_absolute_error,0.053571,6.3125
1,mean_squared_error,0.535714,241.1875
2,median_absolute_error,0.000000,2.0000


### Hyperparameter Tuning

In [ ]:
param_grid = {
    "regressor__n_estimators" : [10, 50, 100, 250],
    "regressor__max_depth" : [None, 3, 5, 10, 15],
    "regressor__min_samples_split" : [2, 5, 7, 10]
}

In [ ]:
search = GridSearchCV(pipe, param_grid, n_jobs = -1, scoring = ['neg_mean_absolute_error',
                                                                'neg_mean_squared_error',
                                                                'neg_median_absolute_error'],
                      refit = 'neg_mean_squared_error', verbose = 2.1 )

In [ ]:
search.fit(X_train, y_train);

Fitting 5 folds for each of 80 candidates, totalling 400 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan]
  category=UserWarning,


In [ ]:
print(search.best_params_)
y_pred_train = search.predict(X_train)
y_pred_test = search.predict(X_test)

results = pd.DataFrame({'Metric (tons)':['mean_absolute_error', 'mean_squared_error', 'median_absolute_error'],
              'Train':[mean_absolute_error(y_train,y_pred_train),
                       mean_squared_error(y_train,y_pred_train),
                       median_absolute_error(y_train,y_pred_train)],
              'Test':[mean_absolute_error(y_test,y_pred_test),
                       mean_squared_error(y_test,y_pred_test),
                       median_absolute_error(y_test,y_pred_test)]})
display(results)

{'regressor__max_depth': None, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 10}


,Metric (tons),Train,Test
0,mean_absolute_error,0.125000,5.241071
1,mean_squared_error,0.839286,85.526786
2,median_absolute_error,0.000000,2.000000


## XGBoost

In [ ]:
xgb_pipe = Pipeline([
    ('preprocessor',preprocessor),
    ('regressor',XGBRegressor(random_state=10302022,
                              n_jobs = -1,
                              objective = 'reg:squarederror'))
])

In [ ]:
xgb_param_grid = {
    "regressor__n_estimators" : [10, 50, 100, 250, 500],
    "regressor__max_depth" : [None, 3, 5, 10, 15],
    "regressor__learning_rate" : [.1, .07, .05, .03, .01, .001],
    "regressor__max_leaves" : [0, 10, 20, 25, 30, 50] #0 is no limit
}

In [ ]:
xgb_search = GridSearchCV(xgb_pipe, xgb_param_grid, n_jobs = -1, scoring = ['neg_mean_absolute_error',
                                                                'neg_mean_squared_error',
                                                                'neg_median_absolute_error'],
                      refit = 'neg_mean_squared_error', verbose = 2.1 )

In [ ]:
xgb_search.fit(X_train, y_train);

Fitting 5 folds for each of 900 candidates, totalling 4500 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
900 fits failed out of a total of 4500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
900 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/sklearn.py", line 396, in fit
    callbacks=callbacks)
  File "/usr/local/lib/python3.7/dist-packages/xgboost/tr

In [ ]:
print(xgb_search.best_params_)
y_pred_train = xgb_search.predict(X_train)
y_pred_test = xgb_search.predict(X_test)

results = pd.DataFrame({'Metric (tons)':['mean_absolute_error', 'mean_squared_error', 'median_absolute_error'],
              'Train':[mean_absolute_error(y_train,y_pred_train),
                       mean_squared_error(y_train,y_pred_train),
                       median_absolute_error(y_train,y_pred_train)],
              'Test':[mean_absolute_error(y_test,y_pred_test),
                       mean_squared_error(y_test,y_pred_test),
                       median_absolute_error(y_test,y_pred_test)]})
display(results)

{'regressor__learning_rate': 0.1, 'regressor__max_depth': 3, 'regressor__max_leaves': 0, 'regressor__n_estimators': 100}


,Metric (tons),Train,Test
0,mean_absolute_error,3.178986,4.890546
1,mean_squared_error,30.099500,75.422166
2,median_absolute_error,1.709862,2.475006


#### Save Model

Assuming you have a fitted model ```knn``` , you can save and load a model like this:


```
# Save the model as a pickle in a file
joblib.dump(knn, 'filename.pkl')
  
# Load the model from the file
knn_from_joblib = joblib.load('filename.pkl')
  
# Use the loaded model to make predictions
knn_from_joblib.predict(X_test)
```



In [ ]:
# Save the model as a pickle in a file

joblib.dump(xgb_search, 'xgb_predict_rotor_weight_tons.pkl')

['/content/drive/MyDrive/Wind Turbine Capstone Project/Sarosh/xgb_predict_rotor_weight_tons.pkl']